상추의 생육 환경 생성 AI 경진대회 왕왕상츄 팀 예측모델 코드입니다.

- 누적값 계산, DAT 곱, 각 변수 간 곱으로 파생변수를 생성했으며
- XGBoostRegreesor 단일 모델에 gridsearch 를 활용해 파라미터를 튜닝했습니다.

## drive 연동

In [1]:
#drive 와 연동시켜줍니다

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## import

In [2]:
! pip install xgboost==0.90
! pip install numpy==1.21.5
! pip install pandas==1.3.5
! pip install scikit-learn==1.0.2
! pip install datetime
! pip install tqdm
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 15.7 MB 27.1 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 52 kB 1.0 MB/s 
     |████████████████████████████████| 261 kB 57.9 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 20 not upgraded.
Need to get 9,604 kB of archives.
After this operation, 29.5 MB of additional disk space will be used.
Get:1 http://archive.ubuntu

In [1]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_validate
from datetime import datetime, timedelta
import copy
import zipfile
import glob
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.ensemble import ExtraTreesRegressor
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns


#전처리

## 1. 환경변수 제한 범위 전처리 함수

In [2]:
# 내부온도 전처리 함수
def preproc_temper(df):
    df['내부온도관측치']=np.where(df['내부온도관측치']<4,4,df['내부온도관측치'])
    df['내부온도관측치']=np.where(df['내부온도관측치']>40,40,df['내부온도관측치'])
    return df
# 내부습도 전처리 함수
def preproc_hum(df):
    df['내부습도관측치']=np.where(df['내부습도관측치']<0,0,df['내부습도관측치'])
    df['내부습도관측치']=np.where(df['내부습도관측치']>100,100,df['내부습도관측치'])
    return df
# co2관측치 전처리 함수
def preproc_co2(df):
    df['co2관측치']=np.where(df['co2관측치']<0,0,df['co2관측치'])
    df['co2관측치']=np.where(df['co2관측치']>1200,1200,df['co2관측치'])
    return df
# ec관측치 전처리 함수
def preproc_ec(df):
    df['ec관측치']=np.where(df['ec관측치']<0,0,df['ec관측치'])
    df['ec관측치']=np.where(df['ec관측치']>8,8,df['ec관측치'])
    return df
# 분무량 전처리 함수
def preproc_spray(df):
    df['시간당분무량']=np.where(df['시간당분무량']<0,0,df['시간당분무량'])
    df['시간당분무량']=np.where(df['시간당분무량']>3000,3000,df['시간당분무량'])
    df['일간누적분무량']=np.where(df['일간누적분무량']>72000, 72000, df['일간누적분무량'])
    return df

# 광량 전처리 함수
def preproc_light(df):
    df['시간당백색광량']=np.where(df['시간당백색광량']>120000, 120000, df['시간당백색광량'])
    df['시간당적색광량']=np.where(df['시간당적색광량']>120000, 120000, df['시간당적색광량'])      
    df['시간당청색광량']=np.where(df['시간당청색광량']>120000, 120000, df['시간당청색광량'])    
    df['일간누적백색광량']=np.where(df['일간누적백색광량']>2880000, 2880000, df['일간누적백색광량']) 
    df['일간누적청색광량']=np.where(df['일간누적청색광량']>2880000, 2880000, df['일간누적청색광량']) 
    df['일간누적적색광량']=np.where(df['일간누적적색광량']>2880000, 2880000, df['일간누적적색광량']) 
    df['시간당총광량']=np.where(df['시간당총광량']>120000, 120000, df['시간당총광량'])
    df['일간누적총광량']=np.where(df['일간누적총광량']>2880000, 2880000, df['일간누적총광량'])    
    return df

## 2. 시간별 변수 -> 일별 변수로 전처리
- 내부온도관측치 : median
- 내부습도관측치: median
- co2관측치 : median
- ec관측치 : median
- 시간당분무량 : median
- 일간누적분무량 : max
- 시간당백색광량 : median
- 일간누적백색광량 : max 
- 시간당적색광량 : median
- 일간누적적색광량 : max
- 시간당청색광량 : median
- 일간누적청색광량 : max
- 시간당총광량 : median
- 일간누적총광량 : max


In [3]:
def f(x):
  d = {}
  d['내부온도관측치'] = x['내부온도관측치'].median()
  d['내부습도관측치'] = x['내부습도관측치'].median()
  d['co2관측치'] = x['co2관측치'].median()
  d['ec관측치'] = x['ec관측치'].median()
  d['시간당분무량'] = x['시간당분무량'].median()
  d['일간누적분무량'] = x['일간누적분무량'].max()
  d['시간당백색광량'] = x['시간당백색광량'].median()
  d['일간누적백색광량'] = x['일간누적백색광량'].max()
  d['시간당적색광량'] = x['시간당적색광량'].median()
  d['일간누적적색광량'] = x['일간누적적색광량'].max()
  d['시간당청색광량'] = x['시간당청색광량'].median()
  d['일간누적청색광량'] = x['일간누적청색광량'].max()
  d['시간당총광량'] = x['시간당총광량'].median()
  d['일간누적총광량'] = x['일간누적총광량'].max()
  d['실내 일교차'] = x['실내 일교차'].median()
  

  return pd.Series(d, index =['내부온도관측치', '내부습도관측치', 'co2관측치',
       'ec관측치', '시간당분무량', '일간누적분무량', '시간당백색광량', '일간누적백색광량', '시간당적색광량',
       '일간누적적색광량', '시간당청색광량', '일간누적청색광량', '시간당총광량', '일간누적총광량','실내 일교차'])

## 3. 파생변수 생성 함수
- 습도_온도
- 습도_온도_CO2
- 비료 : EC관측치 X 일간누적분무량
- 적산온도
- 습도_누적
- co2_누적
- ec_누적
- 광량_누적
- 비료_누적
- co2_ec
- 총누적백색광량
- 총누적적색광량
- 총누적청색광량
- 총누적총광량
- ec_시간당분무량
- ec_일간누적분무량
- ec_분무량_누적


In [4]:
def new_col(particular_case): # 변수 전처리 함수
  group_df = particular_case.groupby('DAT').apply(f) # 날짜에 대하여 groupby 한 함수 생성
  
  new_group_df = group_df  # 날짜에 대하여 groupby 된 groupbydf 에 대하여 평균값으로 치환해 데이터프레임 생성
  # 파생변수
  new_group_df['습도_온도'] = new_group_df['내부온도관측치'] * new_group_df['내부습도관측치'] # 습도_온도 변수
  new_group_df['습도_온도_CO2'] = new_group_df['습도_온도'] * new_group_df['co2관측치'] # 습도_온도_co2
  new_group_df['비료'] = new_group_df['ec관측치']*new_group_df['일간누적분무량'] # 비료
  med=new_group_df['내부온도관측치'].median()
  temp=np.where(new_group_df['내부온도관측치']<med,0,new_group_df['내부온도관측치'])
  new_group_df['적산온도']=temp.cumsum()
  new_group_df['습도_누적'] = new_group_df['내부습도관측치'].cumsum()
  new_group_df['co2_누적'] = new_group_df['co2관측치'].cumsum()
  new_group_df['ec_누적'] = new_group_df['ec관측치'].cumsum()
  new_group_df['광량_누적'] = new_group_df['일간누적총광량'].cumsum()
  new_group_df['co2_ec'] = new_group_df['co2관측치']*new_group_df['ec관측치']
  new_group_df['비료_누적'] = new_group_df['비료'].cumsum()
  new_group_df['총누적백색광량'] = new_group_df['일간누적백색광량'].cumsum()
  new_group_df['총누적적색광량'] = new_group_df['일간누적적색광량'].cumsum()
  new_group_df['총누적청색광량'] = new_group_df['일간누적청색광량'].cumsum()
  new_group_df['총누적총광량'] = new_group_df['일간누적총광량'].cumsum()
  new_group_df['ec_시간당분무량'] = new_group_df['ec관측치']*new_group_df['시간당분무량']
  new_group_df['ec_일간누적분무량'] = new_group_df['ec관측치']*new_group_df['일간누적분무량']
  new_group_df['ec_분무량_누적']= new_group_df['ec_시간당분무량'].cumsum()
  new_group_df = preproc_temper(new_group_df)
  new_group_df = preproc_hum(new_group_df)
  new_group_df = preproc_co2(new_group_df)
  new_group_df = preproc_ec(new_group_df)
  new_group_df = preproc_spray(new_group_df)
  new_group_df = preproc_light(new_group_df)
  new_group_df.drop(['시간당분무량', '시간당백색광량', '시간당적색광량', '시간당청색광량', '습도_온도'], axis=1, inplace=True)



  return new_group_df
  
  
def tem_def(particular_case):
  temp = particular_case
  temp['실내 일교차'] = 0
  tem_def = temp.groupby('DAT')['내부온도관측치'].max() - temp.groupby('DAT')['내부온도관측치'].min()
  for i in range(28):
    temp['실내 일교차'].loc[24*i:24*(i+1)] = tem_def.loc[i]

  return temp  

## 4. train셋 생성하기

### 1- 29 case 의 train 데이터 리스트로 *불러오기*

In [5]:
train_list = []
col_list = []

for case in tqdm(range(1,29)):
  if case <10 :
    case = '0' +str(case)  

  df_case = pd.read_csv(f'/content/drive/MyDrive/2022_상추재배/train_input/CASE_{case}.csv')
  df_case = tem_def(df_case) # 실내 일교차 열 추가 
  group = new_col(df_case) # change_col 함수 적용

  df_target = pd.read_csv(f'/content/drive/MyDrive/2022_상추재배/train_target/CASE_{case}.csv')
  
  group = group.reset_index()
  group = group.sort_values(by='DAT')

  if case =='01':
    col_list = group.columns 
  else:
    group.columns = col_list

  group['target'] = list(df_target['predicted_weight_g'])    

  train_list.append(group)

  0%|          | 0/28 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
  4%|▎         | 1/28 [00:01<00:36,  1.36s/it]/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
  7%|▋         | 2/28 [00:02<00:24,  1.07it/s]/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the

### 리스트 형태의 train 데이터 -> 데이터 프레임으로 변환

In [8]:
def get_data(data_list):
  copy_list = copy.deepcopy(data_list) #원본과 영향 받지 않기 위해 deepcopy 해준다
  new_data = ""
  for i in range(len(copy_list)):
    if i ==0:
      new_data = copy_list[i] # data_list 초기화
    else:
      new_data = pd.concat([new_data, copy_list[i]])  # 0 -> 1 -> 2 하나씩 붙여준다

  return new_data  

In [9]:
train = get_data(train_list) # train_list 에 대하여 하나의 데이터프레임으로 묶어준다

In [10]:
train

,DAT,내부온도관측치,내부습도관측치,co2관측치,ec관측치,일간누적분무량,일간누적백색광량,일간누적적색광량,일간누적청색광량,시간당총광량,...,co2_ec,비료_누적,총누적백색광량,총누적적색광량,총누적청색광량,총누적총광량,ec_시간당분무량,ec_일간누적분무량,ec_분무량_누적,target
0,0,27.119167,75.737500,531.675000,1.391470,3961.27,255677.8594,13698.4344,0.0000,19231.52200,...,739.809554,5511.986444,2.556779e+05,13698.4344,0.0000,2.693763e+05,337.229594,5511.986444,337.229594,0.167719
1,1,27.895000,75.931667,518.283333,1.383575,3980.56,256324.5263,13708.3632,0.0000,19231.52200,...,717.083960,11019.390490,5.120024e+05,27406.7976,0.0000,5.394092e+05,339.764559,5507.404046,676.994153,0.181787
2,2,26.640000,75.680833,537.283333,1.393320,4093.70,255724.2709,13675.2672,0.0000,19068.54300,...,748.607594,16723.224419,7.677267e+05,41082.0648,0.0000,8.088087e+05,342.157583,5703.833928,1019.151736,0.265921
3,3,26.335833,75.645000,804.716667,1.401997,3980.56,256324.5263,13708.3632,0.0000,19231.52200,...,1128.210641,22303.959022,1.024051e+06,54790.4280,0.0000,1.078842e+06,344.288491,5580.734603,1363.440227,0.423650
4,4,26.752500,76.239166,541.016667,1.403245,3935.55,255829.4703,13681.8864,0.0000,19231.52200,...,759.178666,27826.497947,1.279881e+06,68472.3144,0.0000,1.348353e+06,340.083323,5522.538925,1703.523550,0.475272
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23,23,26.881667,58.576667,459.025000,1.173069,27895.87,84543.1884,12856.1412,6038.8575,3669.35270,...,538.468087,377879.041229,4.307065e+06,113155.2240,53190.5075,4.473410e+06,1764.448568,32723.785765,19482.110608,64.875499
24,24,24.530000,59.615833,446.708333,1.190556,3292.69,146626.3049,22339.8000,10503.3825,5786.99725,...,531.831120,381799.171837,4.453691e+06,135495.0240,63693.8900,4.652880e+06,150.010009,3920.130607,19632.120617,74.002614
25,25,24.466667,60.382500,455.916667,1.191734,2995.68,146617.0226,22319.9424,10495.5500,5886.54195,...,543.331295,385369.224906,4.600308e+06,157814.9664,74189.4400,4.832312e+06,150.158457,3570.053070,19782.279074,76.342275
26,26,25.591667,60.102500,429.241667,1.172986,2620.28,146632.4931,22266.9888,10473.6190,5782.32210,...,503.494444,388442.776533,4.746940e+06,180081.9552,84663.0590,5.011685e+06,147.796230,3073.551626,19930.075304,82.621245


###추가 변수 생성 
- ec_분무량_누적*DAT
- 내부습도관측치*일간누적분무량*DAT
- co2관측치*DAT
- 습도_누적*DAT
- 비료_누적*DAT

In [11]:
def make_col(df):
  df['ec_분무량_누적*DAT'] = df['ec_분무량_누적']*df['DAT']
  df['내부습도관측치*일간누적분무량*DAT'] = df['내부습도관측치']*df['일간누적분무량']*df['DAT']
  df['co2관측치*DAT'] =df['co2관측치']*df['DAT']
  df['습도_누적*DAT']= df['습도_누적']*df['DAT']
  df['비료_누적*DAT'] = df['비료_누적']*df['DAT']

  return df

In [12]:
train = make_col(train)

## test셋 생성하기


In [13]:
test_list = []
col_list = []

for case in tqdm(range(1,6)):
  if case <10 :
    case = '0' +str(case)

  df_case = pd.read_csv(f'/content/drive/MyDrive/2022_상추재배/test_input/TEST_{case}.csv')
  df_case = tem_def(df_case) # 실내 일교차 열 추가 

  group = new_col(df_case)
  df_target = pd.read_csv(f'/content/drive/MyDrive/2022_상추재배/test_target/TEST_{case}.csv')
  group = group.reset_index()

  group = group.sort_values(by='DAT')

  if case =='01':
    col_list = group.columns
  else:
    group.columns = col_list

  group['target'] = list(df_target['predicted_weight_g'])    

  test_list.append(group)

  0%|          | 0/5 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
 20%|██        | 1/5 [00:01<00:04,  1.03s/it]/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
 40%|████      | 2/5 [00:01<00:02,  1.36it/s]/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the ca

In [14]:
test = get_data(test_list) # test 데이터 생성

In [15]:
test = make_col(test)

In [16]:
train.columns

Index(['DAT', '내부온도관측치', '내부습도관측치', 'co2관측치', 'ec관측치', '일간누적분무량', '일간누적백색광량',
       '일간누적적색광량', '일간누적청색광량', '시간당총광량', '일간누적총광량', '실내 일교차', '습도_온도_CO2',
       '비료', '적산온도', '습도_누적', 'co2_누적', 'ec_누적', '광량_누적', 'co2_ec', '비료_누적',
       '총누적백색광량', '총누적적색광량', '총누적청색광량', '총누적총광량', 'ec_시간당분무량', 'ec_일간누적분무량',
       'ec_분무량_누적', 'target', 'ec_분무량_누적*DAT', '내부습도관측치*일간누적분무량*DAT',
       'co2관측치*DAT', '습도_누적*DAT', '비료_누적*DAT'],
      dtype='object')

## 실내 일교차, DAT feature drop

In [17]:
train.drop(['실내 일교차'], axis=1, inplace=True)
test.drop(['실내 일교차'], axis=1, inplace=True)

In [18]:
y_train = np.log1p(train['target'])
X = train.drop(['target', 'DAT'],axis=1)
X_test = test.drop(['target', 'DAT'],axis=1)
X_col = X.columns

In [19]:
print(len(X.columns))
sorted(X.columns)

31


['co2_ec',
 'co2_누적',
 'co2관측치',
 'co2관측치*DAT',
 'ec_누적',
 'ec_분무량_누적',
 'ec_분무량_누적*DAT',
 'ec_시간당분무량',
 'ec_일간누적분무량',
 'ec관측치',
 '광량_누적',
 '내부습도관측치',
 '내부습도관측치*일간누적분무량*DAT',
 '내부온도관측치',
 '비료',
 '비료_누적',
 '비료_누적*DAT',
 '습도_누적',
 '습도_누적*DAT',
 '습도_온도_CO2',
 '시간당총광량',
 '일간누적백색광량',
 '일간누적분무량',
 '일간누적적색광량',
 '일간누적청색광량',
 '일간누적총광량',
 '적산온도',
 '총누적백색광량',
 '총누적적색광량',
 '총누적청색광량',
 '총누적총광량']

# 모델링
- 단일 XGBRegressor 이용
- gridsearch 이용해 튜닝한 hyperparameter 활용

In [20]:
import xgboost as xgb
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model

In [21]:
xgbr=xgb.XGBRegressor()

In [22]:
param_grid={
    'learning_rate': np.arange(0.001,1.1,0.05),
    'max_depth':[3,4,5,8,11,13,15],
    'eta':[0.01, 0.05, 0.1],
    'objective' :['reg:squarederror'],
     'eval_metric':['rmse'],
}

In [23]:
ss = ShuffleSplit(test_size=0.2)

In [24]:
grid_xgbr=GridSearchCV(xgbr, return_train_score=True,
                       param_grid=param_grid,
                       cv=ss,
                       verbose=0,
                       scoring='neg_root_mean_squared_error',
                       n_jobs=-1)

In [25]:
scaler = StandardScaler()

scaler.fit(X)
scaled_X_train  = scaler.transform(X)
scaled_X_train  = pd.DataFrame(scaled_X_train , columns=X_col)
scaled_X_test  = scaler.transform(X_test)
scaled_X_test  = pd.DataFrame(scaled_X_test , columns=X_col)

In [26]:
grid_xgbr.fit(scaled_X_train,y_train)

GridSearchCV(cv=ShuffleSplit(n_splits=10, random_state=None, test_size=0.2, train_size=None),
             estimator=XGBRegressor(), n_jobs=-1,
             param_grid={'eta': [0.01, 0.05, 0.1], 'eval_metric': ['rmse'],
                         'learning_rate': array([1.000e-03, 5.100e-02, 1.010e-01, 1.510e-01, 2.010e-01, 2.510e-01,
       3.010e-01, 3.510e-01, 4.010e-01, 4.510e-01, 5.010e-01, 5.510e-01,
       6.010e-01, 6.510e-01, 7.010e-01, 7.510e-01, 8.010e-01, 8.510e-01,
       9.010e-01, 9.510e-01, 1.001e+00, 1.051e+00]),
                         'max_depth': [3, 4, 5, 8, 11, 13, 15],
                         'objective': ['reg:squarederror']},
             return_train_score=True, scoring='neg_root_mean_squared_error')

In [27]:
scores_df = pd.DataFrame(grid_xgbr.cv_results_)

In [28]:
mse = -1*scores_df['mean_test_score'] 
scores_df['rmse_test_score'] = np.sqrt(mse)
mse_train = -1*scores_df['mean_train_score'] 
scores_df['rmse_train_score'] = np.sqrt(mse_train)

In [29]:
scores_df[['rmse_train_score', 'rmse_test_score', 'rank_test_score', 'params']].sort_values(by='rank_test_score')

,rmse_train_score,rmse_test_score,rank_test_score,params
16,0.198696,0.449079,1,"{'eta': 0.01, 'eval_metric': 'rmse', 'learning..."
170,0.198696,0.449079,1,"{'eta': 0.05, 'eval_metric': 'rmse', 'learning..."
324,0.198696,0.449079,1,"{'eta': 0.1, 'eval_metric': 'rmse', 'learning_..."
190,0.172735,0.451175,4,"{'eta': 0.05, 'eval_metric': 'rmse', 'learning..."
344,0.172735,0.451175,4,"{'eta': 0.1, 'eval_metric': 'rmse', 'learning_..."
...,...,...,...,...
309,1.560204,1.565807,457,"{'eta': 0.1, 'eval_metric': 'rmse', 'learning_..."
1,1.560204,1.565807,457,"{'eta': 0.01, 'eval_metric': 'rmse', 'learning..."
308,1.560626,1.566133,460,"{'eta': 0.1, 'eval_metric': 'rmse', 'learning_..."
154,1.560626,1.566133,460,"{'eta': 0.05, 'eval_metric': 'rmse', 'learning..."


In [30]:
grid_xgbr.best_params_

{'eta': 0.01,
 'eval_metric': 'rmse',
 'learning_rate': 0.101,
 'max_depth': 5,
 'objective': 'reg:squarederror'}

In [34]:
xgb_best = grid_xgbr.best_estimator_

In [35]:
result = []
xg_reg = xgb_best
xg_reg.fit(scaled_X_train, y_train)
pred = xg_reg.predict(scaled_X_test)
for k in pred:
    result = np.hstack([result, k])
result

array([0.45342892, 0.75764823, 0.93702739, 1.11975408, 1.47113824,
       1.69830739, 2.02136445, 1.96132874, 2.13423705, 2.32360125,
       2.47681689, 2.66761947, 3.20821261, 2.97258401, 2.89619708,
       3.25415874, 3.22249436, 3.37373543, 3.57727385, 3.62103558,
       4.01856518, 4.16228676, 4.32198429, 4.35609245, 4.52806377,
       4.5286684 , 4.55703545, 4.66004992, 0.22960222, 0.62424719,
       0.89327848, 1.11352301, 1.14214468, 1.37233019, 1.30404139,
       1.38598907, 1.48893642, 1.57889485, 1.84822369, 2.06820178,
       2.33068776, 2.41709661, 2.51461768, 2.78213739, 2.81544924,
       2.85660386, 3.05471635, 3.12705851, 3.04485941, 3.3061111 ,
       3.41653824, 3.81922817, 3.57925749, 4.01231766, 4.0911665 ,
       4.06650734, 0.80575097, 0.85849285, 0.9737348 , 1.03050113,
       1.10511124, 1.30543733, 1.42272437, 1.40510678, 1.66446662,
       1.69565785, 1.83679891, 1.90042591, 2.09964252, 2.07499123,
       2.34336662, 2.30084848, 2.35478568, 2.3048439 , 2.41147

In [36]:
expm1_result = np.expm1(result)
expm1_result = pd.DataFrame(columns=['target'],data=expm1_result)

# 모델 저장

In [ ]:
import pickle
with open ('xg_reg.pkl','wb') as f:
    pickle.dump(xg_reg,f)

# 추출

In [ ]:
base_dir = '/content/drive/MyDrive/2022_상추재배'
os.chdir(base_dir)

In [37]:
total_result = []
for case in tqdm(range(1, 6)):
    if case < 10:
        case = '0'+str(case)
    particular_target = pd.read_csv(f'/content/drive/MyDrive/2022_상추재배/test_target/TEST_{case}.csv')
    particular_target['predicted_weight_g'] = list(expm1_result['target'][:len(particular_target)].values)
    expm1_result = expm1_result.drop([i for i in range(len(particular_target))])
    
    expm1_result = expm1_result.reset_index(drop=True)
    total_result.append(particular_target)
for i in range(len(total_result)):
    total_result[i].to_csv(f'TEST_0{i+1}.csv', index=False)
file_ls = ['TEST_01.csv','TEST_02.csv','TEST_03.csv','TEST_04.csv','TEST_05.csv']

with zipfile.ZipFile("final_submit.zip", 'w') as my_zip:
    for i in file_ls:
        my_zip.write(i)
    my_zip.close()

100%|██████████| 5/5 [00:00<00:00, 188.22it/s]
